In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from tqdm import tqdm

# --- Fixed Dataset class with weighted sampling ---
class AngleDataset(Dataset):
    def __init__(self, df, img_dir, transform=None, is_training=False):
        self.df = df.copy()
        self.img_dir = img_dir
        self.transform = transform
        self.is_training = is_training

        # Ensure angles are within [0, 360)
        self.df['angle'] = self.df['angle'] % 360
        
        # Calculate weights for sampling (for handling class imbalance if any)
        if is_training:
            angle_bins = np.linspace(0, 360, 37)  # 36 bins of 10 degrees each
            self.df['angle_bin'] = pd.cut(self.df['angle'], bins=angle_bins, labels=False)
            
            # Fix: Handle NaN values in angle_bin
            self.df['angle_bin'] = self.df['angle_bin'].fillna(0).astype(int)
            
            bin_counts = self.df['angle_bin'].value_counts().sort_index()
            
            # Fix: Make sure all bin indices are in the bin_counts
            all_bins = np.arange(len(angle_bins)-1)
            for bin_idx in all_bins:
                if bin_idx not in bin_counts.index:
                    bin_counts[bin_idx] = 1  # Add a small count to avoid division by zero
            
            bin_counts = bin_counts.sort_index()
            self.weights = 1.0 / bin_counts[self.df['angle_bin'].values].values
            self.weights = torch.FloatTensor(self.weights)
        else:
            self.weights = None

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        angle = row['angle']
        # Convert angle to radians
        angle_rad = angle * np.pi / 180
        # Create target vector [sin(θ), cos(θ)]
        sin_angle = np.sin(angle_rad)
        cos_angle = np.cos(angle_rad)
        
        # Add region_encoded to the return
        region = torch.tensor(row['region_encoded'], dtype=torch.long)
        
        return image, torch.tensor([sin_angle, cos_angle], dtype=torch.float32), torch.tensor(angle, dtype=torch.float32), region

# --- Advanced angular error metrics ---
def mean_absolute_angular_error(y_true, y_pred):
    diff = torch.abs(y_true - y_pred)
    return torch.mean(torch.min(diff, 360 - diff))

def angular_rmse(y_true, y_pred):
    diff = torch.abs(y_true - y_pred)
    min_diff = torch.min(diff, 360 - diff)
    return torch.sqrt(torch.mean(min_diff ** 2))

# --- Improved Angular loss functions ---
def angular_vector_loss(pred, true_angle):
    # Convert true angle to radians
    true_rad = true_angle * np.pi / 180
    # Create true vector [sin(θ), cos(θ)]
    true_vec = torch.stack([torch.sin(true_rad), torch.cos(true_rad)], dim=1)
    # Normalize prediction vector
    pred = F.normalize(pred, dim=1)
    # Compute loss as 1 - dot product
    loss = torch.mean(1 - torch.sum(pred * true_vec, dim=1))
    return loss

def combined_angular_loss(pred, true_angle, alpha=0.8):
    # Vector loss
    vec_loss = angular_vector_loss(pred, true_angle)
    
    # Convert predictions back to angles for direct angle loss
    pred = F.normalize(pred, dim=1)
    pred_angle = (torch.atan2(pred[:, 0], pred[:, 1]) * 180 / np.pi) % 360
    
    # Direct angle loss using minimum distance accounting for circularity
    diff = torch.abs(pred_angle - true_angle)
    min_diff = torch.min(diff, 360 - diff)
    direct_loss = torch.mean(min_diff)
    
    # Combined loss
    return alpha * vec_loss + (1 - alpha) * direct_loss / 180.0

# --- Improved Model Architecture ---
class ImprovedAngleModel(nn.Module):
    def __init__(self, num_regions, backbone_name='efficientnet_b0'):
        super(ImprovedAngleModel, self).__init__()
        
        # Choose backbone based on parameter
        if backbone_name == 'resnet50':
            self.backbone = models.resnet50(pretrained=True)
            self.backbone.fc = nn.Identity()  # Remove final FC layer
            self.feature_dim = 2048
        elif backbone_name == 'efficientnet_b0':
            self.backbone = models.efficientnet_b0(pretrained=True)
            self.backbone.classifier = nn.Identity()  # Remove classifier
            self.feature_dim = 1280
        elif backbone_name == 'convnext_small':
            self.backbone = models.convnext_small(pretrained=True)
            self.backbone.classifier[2] = nn.Identity()  # Remove classifier
            self.feature_dim = 768
        else:  # Default to ResNet18
            self.backbone = models.resnet18(pretrained=True)
            self.backbone.fc = nn.Identity()  # Remove final FC layer
            self.feature_dim = 512
        
        # Region embedding
        self.region_embedding = nn.Embedding(num_regions, 128)
        
        # Attention mechanism for feature fusion
        self.attention = nn.Sequential(
            nn.Linear(self.feature_dim + 128, 256),
            nn.ReLU(),
            nn.Linear(256, 1)
        )
        
        # Final layers with multiple branches
        self.shared_fc = nn.Sequential(
            nn.Linear(self.feature_dim + 128, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        
        self.angle_fc = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 2)  # Output sin(θ), cos(θ)
        )
    
    def forward(self, x, regions):
        # Extract image features
        img_features = self.backbone(x)
        
        # Get region embeddings
        region_features = self.region_embedding(regions)
        
        # Concatenate features
        combined_features = torch.cat([img_features, region_features], dim=1)
        
        # Apply attention mechanism
        attention_weights = torch.sigmoid(self.attention(combined_features))
        attended_features = combined_features * attention_weights
        
        # Shared layers
        shared_features = self.shared_fc(attended_features)
        
        # Angle prediction branch
        angle_out = self.angle_fc(shared_features)
        
        return angle_out

# --- Modified transforms without rotation, flip, or noise augmentation ---
def get_transforms(img_size=256, crop_size=224):
    # Training transforms with limited augmentation (no rotation, flip, or noise)
    train_transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
        transforms.RandomResizedCrop(224, scale=(0.9, 1.0)),
        transforms.GaussianBlur(3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.RandomErasing(p=0.5, scale=(0.02, 0.05), ratio=(0.3, 3.3)),
    ])
    
    # Validation transforms
    val_transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    return train_transform, val_transform

# --- Setup paths ---
dataset_folder = '/kaggle/input/smai-project/'
train_csv_path = dataset_folder + 'labels_train.csv'
val_csv_path = dataset_folder + 'labels_val.csv'
train_img_dir = dataset_folder + 'images_train/images_train'
val_img_dir = dataset_folder + 'images_val/images_val'

# Path for predicted regions
regions_path = '/kaggle/input/predicted-regions/predicted_regions/'
train_regions_path = regions_path + 'predicted_regions_train.csv'
val_regions_path = regions_path + 'predicted_regions_val.csv'

# --- Load CSVs ---
train_df = pd.read_csv(train_csv_path)
val_df = pd.read_csv(val_csv_path)

# Load region predictions
predicted_region_train_df = pd.read_csv(train_regions_path)
predicted_region_val_df = pd.read_csv(val_regions_path)

# Drop original Region_ID if it exists
if 'Region_ID' in train_df.columns:
    train_df = train_df.drop('Region_ID', axis=1)
if 'Region_ID' in val_df.columns:
    val_df = val_df.drop('Region_ID', axis=1)

# Merge train data with predicted regions
train_df = pd.merge(train_df, predicted_region_train_df[['filename', 'Region_ID']], 
                   on='filename', how='inner')

# Merge validation data with predicted regions
val_df = pd.merge(val_df, predicted_region_val_df[['filename', 'Region_ID']], 
                 on='filename', how='inner')

# Encode regions
region_encoder = LabelEncoder()
train_df['region_encoded'] = region_encoder.fit_transform(train_df['Region_ID'])
val_df['region_encoded'] = region_encoder.transform(val_df['Region_ID'])

# Get number of unique regions
NUM_REGIONS = len(region_encoder.classes_)
print(f"Number of unique regions: {NUM_REGIONS}")

# --- Model training function ---
def train_and_validate(model, train_loader, val_loader, optimizer, scheduler, num_epochs, device, patience=15):
    best_val_error = float('inf')
    patience_counter = 0
    history = {'train_loss': [], 'val_maae': []}
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        
        for images, angle_vectors, angles, regions in tqdm(train_loader, desc=f'Epoch {epoch+1} Training'):
            images = images.to(device)
            angle_vectors = angle_vectors.to(device)
            angles = angles.to(device)
            regions = regions.to(device)
            
            optimizer.zero_grad()
            outputs = model(images, regions)
            loss = combined_angular_loss(outputs, angles, alpha=0.7)
            loss.backward()
            
            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            if scheduler is not None:
                scheduler.step()  # Step the scheduler each batch
            
            train_loss += loss.item() * images.size(0)
        
        avg_train_loss = train_loss / len(train_loader.dataset)
        history['train_loss'].append(avg_train_loss)
        
        # --- Validation ---
        model.eval()
        true_angles = []
        pred_angles = []
        
        with torch.no_grad():
            for images, angle_vectors, angles, regions in tqdm(val_loader, desc=f'Epoch {epoch+1} Validation'):
                images = images.to(device)
                angles = angles.to(device)
                regions = regions.to(device)
                
                outputs = model(images, regions)
                outputs = F.normalize(outputs, dim=1)
                
                # Convert sin(θ), cos(θ) predictions back to angles
                sin_preds = outputs[:, 0]
                cos_preds = outputs[:, 1]
                angle_preds = (torch.atan2(sin_preds, cos_preds) * 180 / np.pi) % 360
                
                true_angles.append(angles.cpu())
                pred_angles.append(angle_preds.cpu())
        
        true_all = torch.cat(true_angles)
        preds_all = torch.cat(pred_angles)
        val_error = mean_absolute_angular_error(true_all, preds_all).item()
        history['val_maae'].append(val_error)
        
        print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val MAAE: {val_error:.4f}')
        
        # Save best model
        if val_error < best_val_error:
            best_val_error = val_error
            torch.save(model.state_dict(), 'best_model.pth')
            print(f'✅ Best model saved with Val MAAE: {best_val_error:.4f}')
            patience_counter = 0
        else:
            patience_counter += 1
            print(f'No improvement for {patience_counter} epochs...')
        
        # Early stopping
        if patience_counter >= patience:
            print(f'Early stopping triggered after {epoch+1} epochs')
            break
    
    return history, best_val_error

# --- Main training code ---
def main():
    # Set seeds for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')
    
    # Get transforms
    train_transform, val_transform = get_transforms(img_size=288, crop_size=256)
    
    # Create datasets
    train_dataset = AngleDataset(train_df, train_img_dir, train_transform, is_training=True)
    val_dataset = AngleDataset(val_df, val_img_dir, val_transform)
    
    # Create weighted sampler for training to balance angle distribution
    try:
        sampler = torch.utils.data.WeightedRandomSampler(
            weights=train_dataset.weights,
            num_samples=len(train_dataset),
            replacement=True
        )
        
        # Create dataloaders with sampler
        train_loader = DataLoader(
            train_dataset, 
            batch_size=32, 
            sampler=sampler,
            num_workers=4,
            pin_memory=True
        )
    except Exception as e:
        print(f"Error creating weighted sampler: {e}")
        print("Falling back to regular DataLoader without weighted sampling")
        
        # Create regular dataloader without sampler
        train_loader = DataLoader(
            train_dataset, 
            batch_size=32, 
            shuffle=True,
            num_workers=4,
            pin_memory=True
        )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=32, 
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )
    
    # Initialize model
    backbone_name = 'efficientnet_b0'  # Try different backbones
    model = ImprovedAngleModel(NUM_REGIONS, backbone_name=backbone_name).to(device)
    
    # Use AdamW optimizer with weight decay
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
    
    # Learning rate scheduler
    num_epochs = 80
    steps_per_epoch = len(train_loader)
    
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=5e-4,
        epochs=num_epochs,
        steps_per_epoch=steps_per_epoch,
        pct_start=0.1,
        anneal_strategy='cos',
        div_factor=25.0,
        final_div_factor=1e4
    )
    
    # Train model
    history, best_maae = train_and_validate(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        optimizer=optimizer,
        scheduler=scheduler,
        num_epochs=num_epochs,
        device=device,
        patience=15
    )
    
    # Load best model for final evaluation
    model.load_state_dict(torch.load('best_model.pth'))
    model.eval()
    
    # Get predictions and true values for final evaluation
    filenames = []
    true_angles = []
    predictions = []
    
    with torch.no_grad():
        for i, (images, angle_vectors, angles, regions) in enumerate(tqdm(val_loader, desc='Generating predictions')):
            images = images.to(device)
            regions = regions.to(device)
            
            outputs = model(images, regions)
            outputs = F.normalize(outputs, dim=1)
            
            # Convert sin(θ), cos(θ) predictions back to angles
            sin_preds = outputs[:, 0]
            cos_preds = outputs[:, 1]
            angle_preds = (torch.atan2(sin_preds, cos_preds) * 180 / np.pi) % 360
            
            # Extract batch filenames
            batch_indices = range(i * val_loader.batch_size, 
                              min((i + 1) * val_loader.batch_size, len(val_dataset)))
            batch_filenames = [val_df.iloc[j]['filename'] for j in batch_indices]
            filenames.extend(batch_filenames[:len(angles)])  # Match length to actual batch
            
            true_angles.extend(angles.cpu().numpy())
            predictions.extend(angle_preds.cpu().numpy())
    
    # Calculate final MAAE
    final_maae = np.mean(np.minimum(np.abs(np.array(predictions) - np.array(true_angles)), 
                                 360 - np.abs(np.array(predictions) - np.array(true_angles))))
    print(f'Final Mean Absolute Angular Error on validation set: {final_maae:.4f}')
    
    # Save predictions to CSV
    results_df = pd.DataFrame({
        'filename': filenames[:len(true_angles)],  # Ensure same length
        'true_angle': true_angles,
        'predicted_angle': predictions
    })
    results_df.to_csv('angle_predictions.csv', index=False)

if __name__ == "__main__":
    main()

Number of unique regions: 15
Using device: cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 142MB/s] 
Epoch 1 Validation: 100%|██████████| 12/12 [00:02<00:00,  5.39it/s]


Epoch 1, Train Loss: 0.8037, Val MAAE: 78.3973
✅ Best model saved with Val MAAE: 78.3973


Epoch 2 Validation: 100%|██████████| 12/12 [00:01<00:00,  7.54it/s]


Epoch 2, Train Loss: 0.7046, Val MAAE: 68.8643
✅ Best model saved with Val MAAE: 68.8643


...

Epoch 79, Train Loss: 0.0136, Val MAAE: 24.9037
No improvement for 10 epochs...


Epoch 80 Validation: 100%|██████████| 12/12 [00:01<00:00,  6.85it/s]
/tmp/ipykernel_19/492793214.py:418: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(

Epoch 80, Train Loss: 0.0134, Val MAAE: 24.7358
No improvement for 11 epochs...


Generating predictions: 100%|██████████| 12/12 [00:01<00:00,  7.73it/s]

Final Mean Absolute Angular Error on validation set: 24.3993
